# Setup AWS Credentials in EKS cluster

AWS credentials are required to save a model in S3. These credentials are stored in the EKS cluster as a Kubernetes secret.

Create an IAM user `s3user`, attach the S3 access policy, and retrieve temporary credentials

In [1]:
%%bash

source ~/.bash_profile

aws iam create-user --user-name s3user
aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

{
    "User": {
        "Path": "/",
        "UserName": "s3user",
        "UserId": "AIDAQ24BP2I6MJI65NDB5",
        "Arn": "arn:aws:iam::057716757052:user/s3user",
        "CreateDate": "2020-08-01T00:27:23Z"
    }
}


In [2]:
%%bash

source ~/.bash_profile

aws iam create-access-key --user-name s3user > /tmp/create_output.json

# Create `aws-secret` to Allow Kubernetes/Kubeflow to Access Amazon SageMaker

In [3]:
%%bash

source ~/.bash_profile

export AWS_ACCESS_KEY_ID_VALUE=$(jq -j .AccessKey.AccessKeyId /tmp/create_output.json | base64)
echo "export AWS_ACCESS_KEY_ID_VALUE=${AWS_ACCESS_KEY_ID_VALUE}" | tee -a ~/.bash_profile

export AWS_SECRET_ACCESS_KEY_VALUE=$(jq -j .AccessKey.SecretAccessKey /tmp/create_output.json | base64)
echo "export AWS_SECRET_ACCESS_KEY_VALUE=${AWS_SECRET_ACCESS_KEY_VALUE}" | tee -a ~/.bash_profile

#### Apply to EKS cluster.

cat <<EOF | kubectl apply --namespace anonymous -f -
apiVersion: v1
kind: Secret
metadata:
  name: aws-secret
type: Opaque
data:
  AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID_VALUE
  AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY_VALUE
EOF

#### Add the secret to the `kubeflow` namespace, as well.  This is needed until KF Pipelines support namespaces.
cat <<EOF | kubectl apply --namespace kubeflow -f -
apiVersion: v1
kind: Secret
metadata:
  name: aws-secret
type: Opaque
data:
  AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID_VALUE
  AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY_VALUE
EOF


export AWS_ACCESS_KEY_ID_VALUE=QUtJQVEyNEJQMkk2UFc3QUVGSEk=
export AWS_SECRET_ACCESS_KEY_VALUE=UldteVpQdXBkUDhwYWhHV01pOXU5QVR5dUc1bGNoR0VHemtSc3JlWA==
secret/aws-secret configured


Error from server (NotFound): error when creating "STDIN": namespaces "anonymous" not found


# Add More Roles/Policies for Kubernetes/Kubeflow to Access SageMaker

In [7]:
%%bash

source ~/.bash_profile

TRUST="{ \"Version\": \"2012-10-17\", \"Statement\": [ { \"Effect\": \"Allow\", \"Principal\": { \"Service\": \"sagemaker.amazonaws.com\" }, \"Action\": \"sts:AssumeRole\" } ] }"
# aws iam create-role --role-name workshop-sagemaker-kfp-role --assume-role-policy-document "$TRUST"
aws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess

export SAGEMAKER_ROLE_ARN=$(aws iam get-role --role-name workshop-sagemaker-kfp-role --output text --query 'Role.Arn')
echo "export SAGEMAKER_ROLE_ARN=${SAGEMAKER_ROLE_ARN}" | tee -a ~/.bash_profile

cat <<EoF > sagemaker-invoke.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}
EoF

aws iam put-role-policy --role-name workshop-sagemaker-kfp-role --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-invoke.json
aws iam put-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-invoke.json

export SAGEMAKER_ROLE_ARN=arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


CalledProcessError: Command 'b'\nsource ~/.bash_profile\n\nTRUST="{ \\"Version\\": \\"2012-10-17\\", \\"Statement\\": [ { \\"Effect\\": \\"Allow\\", \\"Principal\\": { \\"Service\\": \\"sagemaker.amazonaws.com\\" }, \\"Action\\": \\"sts:AssumeRole\\" } ] }"\n# aws iam create-role --role-name workshop-sagemaker-kfp-role --assume-role-policy-document "$TRUST"\naws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess\naws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess\n\nexport SAGEMAKER_ROLE_ARN=$(aws iam get-role --role-name workshop-sagemaker-kfp-role --output text --query \'Role.Arn\')\necho "export SAGEMAKER_ROLE_ARN=${SAGEMAKER_ROLE_ARN}" | tee -a ~/.bash_profile\n\ncat <<EoF > sagemaker-invoke.json\n{\n    "Version": "2012-10-17",\n    "Statement": [\n        {\n            "Effect": "Allow",\n            "Action": [\n                "sage

In [5]:
!cat sagemaker-invoke.json

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}


# Add More Roles/Policies for Kubernetes/Kubeflow to Access S3

In [6]:
%%bash

source ~/.bash_profile

aws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


CalledProcessError: Command 'b'\nsource ~/.bash_profile\n\naws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess\n'' returned non-zero exit status 2.